# Engineering Requirements
Now that we've defined the roles and bond lifecycle phases, we can create formal mathematical definitions and requirements for the ecosystem. We will take the roles identified and categorize them into classes of agents based on the types of actions that they can take. We will also define the system state variables, mechanisms, and initialization/settlement conditions.   

## Assumptions
#### Assumption 1
The initial value for Alpha is pre-calculated externally and then set upon initialization of the bond.


## Agents

#### Definition 1
Let A be the set of all possible addresses that interact within the system. At any timestep, the set of addresses that exist is $A \subset \lambda$

The address $a \subset \lambda$ is called an **agent** when referring to an address which takes an action. Agent decisions and actions take place over time $t \in Z^{v\geq 0}$ such that at any timestep $t$, an agent can perform an array of agent actions represented as an ordered list of transactions. 


### Agent Class 1: Trader Agents
Trader Agents can execute the *Bond-to-Mint* and *Burn-to-Withdraw* mechanisms. <br>

Constitutes of roles:
1. InvestmentAgent
 
##### Operational Requirement 1
Trader Agents can call the Bond Action and the Burn Action. 
##### Operational Requirement 2
An agent which is able to execute the *Bond-to-Mint* or *Burn-to-Withdraw* mechanisms - such as Trader Agents - must not be involved in the Claim Evaluation, Claim Auditing, and Claim Resolution stages of the *Claim* mechanism as this creates a conflict of interest. 

### Agent Class 2: Claim & Dispute Issuers
Claim & Dispute Issuers can execute the Claim Submission and Dispute Submission functions of the *Claim* mechanism. <br>

Constitutes of roles:
1. InvestmentAgent 
2. ServiceAgent 
3. BondIssuer 
4. ProjectOwner 
5. OutcomesPayer 
 
##### Operational Requirement 3
Claim & Dispute Issuers call the SubmitClaim Action or SubmitDispute Action
##### Operational Requirement 4
Claim & Dispute Issuers are explicity excluded from having the ability to call EvaluateClaim Action, AuditClaim Action, or ResolveClaim Action. It would produce a conflict of interest if those that issue claims and disputes could also participate in evaluating, auditing, or resolving them.
 
### Agent Class 3: Evaluators
Claim Evaluators can execute the Claim Evaluation, Claim Auditing, and Claim Resolution functions of the *Claim* mechanism. <br>

Constitutes of roles:
1. VerificationAgent
2. Administrator
3. Auditor
4. Arbitrator
 
##### Operational Requirement 5
Evaluators can call the EvaluateClaim Action, AuditClaim Action, or ResolveClaim Action to process Claims, or the EvaluateDispute Action or ResolveDispute Action to process Disputes
##### Operational Requirement 6
Agents involved in the Claim Evaluation, Claim Auditing, and Claim Resolution stages of the *Claim* mechanism, such as Claim & Dispute Evaluators, are explictly excluded from holding or trading bond tokens as it results in a conflict of interest. 

### Agent Class 4: Outcomes Payers
Outcomes Payers evaluate the state of the project through the CompleteBond Action and  disburse the final payout to Bond token holders through the SettleBond Action, all according to the project terms. <br>

Constitutes of roles:
1. OutcomesPayers
 
##### Operational Requirement 7
OutcomesPayers can perform the CompleteBond Action and SettleBond Action. 
##### Operational Requirement 8
OutcomesPayers are explicitly excluded from the ability to execute the EvaluateClaim Action, AuditClaim Action, ResolveClaim Action, EvaluateDispute Action and ResolveDispute Action.
##### Operational Requirement 9
OutcomesPayers are disincentivized from holding bond tokens.

Since an OutcomesPayer makes the final decision evaluating if the bond or project terms satisfy the conditions for payout to InvestmentAgents and ProjectOwners, holding bond tokens presents a conflict of interest.

## System Boundary 
The system under consideration is at the bond system level. The level above the bond system consists of the project system, which accounts for all of the activities that take place on the project. Although the focus of this design is specifically on the bond system, the interface at the boundary and the flows of information and funds between project and bond must be considered. One more level out there is again the notion of a bond portfolio, which is comprised of multiple bonds. 

![system boundary diagram](artifacts/SystemBoundaryV1.png)

## System Requirements 

#### System Requirement 1 
The state of all Claims issued in a bond must reflect the state of the bonding curve. If the bonding curve state is not respected, Disputes will be issued against the Claims to regain the bonding curve state. 

The Dispute mechanism acts as a fail-safe, therefore making the protocol more resilient.

#### System Requirement 2 
The economic incentive layer of the system, which contains all the bonding curve mechanisms, must incentivize behaviour that achieves system goals. 

#### System Requirement 3
All agent roles have restrictions over their action space, which are defined in the role's corresponding operational requirements.

## System State Variables

#### Definition 2
The **timestep** or period $t$ defined for the system is one block. A block contains an ordered list of transactions, also referred to as agent actions $u_t$. Therefore, in each timestep, an agent $a$ can perform multiple actions chosen from the set of all legal actions $U$.


Note that some tests or simulations may call for a finer timestep granularity, for example 1 timestep = 1 transaction. At this granularity, only one agent action could be performed at a single timestep.

#### Definition 3
The **agent-level state** represents all agent states at a given time $t$. The agent state is a vector $\hat{x}_{a,t}$ making the agent state space $\hat{X}_{a} \in R^k$ such that $\forall a, \forall t, \hat{x}_{a,t} \in \hat{X}_a$. Since the agent's state transition reflects an agent action, the agent-level state summarizes the flow of information in the system indexed by time. The agent-level state is given by

$$     (\hat{x}_{1,t}, \hat{x}_{2,t}, \ldots, \hat{x}_{n,t})\in\prod_{a=1}^n\hat{X}_a \subseteq \mathbb{R}^{nk}.
$$

#### Definition 4
The **system state** is the network’s internal state composed of a finite number of elements, denoted by $\bar{x}_t$. The system-level state depends on the information arrival process summarized by time $t$. The system-level state space is a set $\bar{X} \in R^m$, such that $\forall t$, $\bar{x}_t \in \bar{X}$ where $m$ is the number of finite elements. The system state $x_t$ is the state of all agents and the system-level state, and is given by

$$
x_t := (\hat{x}_{1,t}, \hat{x}_{2,t}, \ldots, \hat{x}_{n,t}, \bar{x}_t) \in X := \prod_{a=1}^n\hat{X}_a \times \bar{X} \subseteq \mathbb{R}^{nk} \times \mathbb{R}^m.
$$

#### Definition 5
The **reserve** $R_t \in \mathbb{R}_{++}$ at time $t$ is the total quantity of external currency bonded to the bonding curve contract.

At time $t$, each agent $a$ possess their holding of Bond Tokens, denoted by $r_{a,t}>0$

#### Definition 6
The **supply** $S_t \in \mathbb{R}_{++}$ at time $t$ is the total quantity of bond tokens issued by the bonding curve contract.

At time $t$, the local state of agent $a$ comprises of the individual holding of bond tokens, $s_{a,t}$.

#### Definition 7
The **price** signal $P_t \in \mathbb{R}_{++}$ at time $t$ is an estimate of the value of the bond token, in units of $R$ per units of $S$.

#### Definition 8
The **Alpha** state variable $\alpha \in [0,1]$ is an estimate of the probability of project success. It is represented normalized such that $\alpha=0$ indicates that the project is estimated to fail, and $\alpha=1$ indicates the highest likelhood of success.


#### Defintion 10
The **ProjectTime** state variable $p \in \mathbb{Z}_{++}$ describes the duration for which the project has been in the Execution phase. 


#### Defintion 11

The **Reserve Ratio** $\rho$  of a bond is the ratio of the value that is bonded to the bonding curve reserve to the total value of bond tokens issued (price * value of one bond token). It is given by $$\rho = \frac{R}{PS}$$

#### Definition 12 (Not used)

## Initialization Conditions
The ProjectTime and Alpha state variables need to meet a pre-specified criteria for the state to transition into the Execution phase. In the bonding curve use case, the initialization conditions are also referred to as launch conditions.

#### Definition 13
The **ProjectTime threshold** $p_{lim}$ describes the maximum allowable duration for a project's Execution phase. 

At initialization, $p_{lim}$ is set to a strictly positive finite integer $$p_{lim} \in \mathbb{Z}_{++}$$ $$p \overset{set}{=} 0$$

#### Definition 14
The **Alpha threshold** is $\alpha_{lim}$ describes the minimum alpha value required for the settlement to be successful. $\alpha_{lim}$ is set to a positive finite integer during project initialization.

At initialization, $\alpha_{lim}$ is set to a real value between 0 and 1 $$\alpha_{lim} \in [0,1]$$

## Settlement Conditions

#### Condition 1
For successful settlement, the project must complete execution before the duration specified by the ProjectTime threshold. If the project does not reach Settlement Phase before $p_{lim}$, the project fails settlement and is terminated. 

For successful settlement, $$p \overset{!}{\geq} p_{lim}$$


#### Condition 2
For successful settlement, the project's Alpha must be evaluated to be greater than or equal to $\alpha_{lim}$ set at initialization. Failing to meet this criteria results in the failure of settlement and project termination.

For successful settlement, $$\alpha \overset{!}{\geq} \alpha_{lim}$$

![settlement phase diagram](artifacts/SettlementConsiderationPhasev1.png)

## Mechanisms 

#### Definition 15
The set of **mechanisms** is $F$ such that any $f \in F$ is an operator $$f : \mathcal{X} \times U \to \mathcal{X}$$
where $\mathcal{X}$ is the space of all possible states $X$, and $U$ is the space of all legal actions associated with the mechanism $f$.

#### Functional Requirement 1
The bonding curve mechanism should be robust enough to account for large or catastropic risks that occur at either extemes of the curve. 

[comment]: # (Define a cut off point for asymptotic portion of the curve)

#### Mechanism 1 
The **bond-to-mint** mechanism mints bond tokens in exchange of external currency through an agent’s action $u_{a, t}$ . The agent’s action represents a transaction to transfer $\Delta R_t := r_{a,t} - r_{a, t+1} \geq 0$ quantity of external currency into the bonding curve system. Quantity $\Delta R_t$ of external currency is transferred into the bonding curve, resulting in 

System state, $x_{t+1}$

$$R_{t+1} = R_{t} + \Delta R_t$$


Agent-level state, $\hat x_{t+1}$

$$r_{a, t+1} = r_t - \Delta R_t$$


#### Functional Requirement 2
Upon the execution the Bond Action, bond tokens are added to the bonding curve. 

#### Mechanism 2 
The **burn-to-withdraw** mechanism removes bond tokens to redeem external currency through an agent’s action $u_{a, t}$ . The agent’s action represents a transaction to remove $\Delta S_t := s_{a, t
} - s_{a, t+1} \leq 0$ quantity of bond tokens from the bonding curve system. Quantity $\Delta S_t$ of bond tokens are removed from the bonding curve, resulting in

System state, $x_{t+1}$
$$S_{t+1} = S_t + \Delta S_t$$

Agent-level state, $\hat x_{t+1}$
$$s_{a, t+1} = s_{a,t} - \Delta S_t$$


#### Functional Requirement 3
Upon the execution of the Burn Action, bond tokens are removed from the bonding curve.

#### Mechanism 3
The **claim** mechanism involves the lifecycle of claims. Claims undergo the stages of submission, evaluation, audit, and resolution. Claims Submission: claims are submitted to prove impact or progress towards achieving the project's predetermined outcomes. Claim Evaluation: submitted claims go through a process of evaluation to verify their validity. Claim Auditing: submitted claims may be audited. Claim Resolution: claims are resolved and completed. During resolution, a dispute could be submitted which would result in subsequent dispute evaluation and resolution.

![](https://i.imgur.com/vamnLGV.png)

#### Functional Requirement 4
A lack of trading activity on the bonding curve does not indicate a degradation of the Bond Alpha, Project Alpha, or the bonding curve itself. 

<!-- [comment]: # (Sustainability: Consider temporal effects on investor sentiment, i.e passage of time correlates with decaying excitement about project, thus lesser activity. However, sunk cost bias simultaneously is in contention with this.) -->

#### Definition 16
The bonding curve **conservation function** describes a conserved quantity, which is a functional relationship between the reserve and supply tokens, and is given by 

$$V(R_t,S_t)  := \frac{S_t^\kappa}{R_t} \equiv V_0,$$

where $V_0 = V(R_0,S_0) := \frac{S_0^\kappa}{R_0}$ is a constant defined by initial supply $S_0$ and initial reserve $R_0$. Parameter $\kappa$ is the curvature of the bonding curve.

This conservation function imposed over the bonding curve ensures that the price of the token reflects the amount invested into projects in the platform, thus preventing imbalances through incentive design.